In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tqdm
%matplotlib inline

In [ ]:
fname = 'test_images/lens_chessboard.png'
nx = 10
ny = 6

fname = 'camera_cal/calibration3.jpg'
nx = 9
ny = 6
img = cv2.imread(fname)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

# If found, draw corners
if ret == True:
    # Draw and display the corners
    cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
    plt.imshow(img)
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    src = corners[[0, nx-1, -nx, -1]].squeeze()
    dst = np.zeros_like(src)
#     topLeft = src[0]
#     botRight = src[-1]
#     dst[0, 0] = dst[2, 0] = topLeft[0]
#     dst[0, 1] = dst[1, 1] = topLeft[1]
#     dst[1, 0] = dst[3, 0] = botRight[0]
#     dst[2, 1] = dst[3, 1] = botRight[1]

    l = min(src[:, 0]); r = max(src[:, 0])
    t = min(src[:, 1]); b = max(src[:, 1])
    dst[0, 0] = dst[2, 0] = l
    dst[0, 1] = dst[1, 1] = t
    dst[1, 0] = dst[3, 0] = r
    dst[2, 1] = dst[3, 1] = b
#     plt.gca().scatter(src[:, 0], src[:, 1], s=1000, color='red')
    plt.gca().scatter(dst[:, 0], src[:, 1], s=1000, color='red')

In [ ]:
import glob

In [ ]:
imgPaths = glob.glob('huginn/IMG*.jpg')

In [ ]:
imgPaths = glob.glob('camera_cal/calibration*.jpg')

In [ ]:
class Undistorter(object):
    
    def __init__(self, nx=7, ny=9):
        self.nx = nx
        self.ny = ny
        
        self.singleObjP = np.zeros((nx*ny, 3), np.float32)
        self.singleObjP[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
        
        self.imgp = []
        
    def fitImg(self, img):
        if isinstance(img, str):
            img = cv2.imread(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self.imageShape = gray.shape[::-1]
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ret, corners = cv2.findChessboardCorners(gray, (self.nx, self.ny), None)
        if ret:
            self.imgp.append(corners)
            
    def fit(self, imgs):
        for img in tqdm.tqdm_notebook(imgs, unit='frame'):
            self.fitImg(img)
        print('Calibrating camera from found points', end=' ... ')
        self.calcParams()
        print('done.')
        
    def calcParams(self):
        objp = [self.singleObjP] * len(self.imgp)
        self.ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv2.calibrateCamera(
            objp, self.imgp, self.imageShape, None, None
        )
        
    def __call__(self, img):
        if isinstance(img, str):
            img = cv2.imread(img)
        return cv2.undistort(img, self.mtx, self.dist, None, self.mtx)

In [ ]:
undistorter = Undistorter(nx=9, ny=6)
undistorter.fit(imgPaths[:15])

In [ ]:
# plt.imshow(cv2.imread('camera_cal/calibration1.jpg'))

In [ ]:
for imgPath in tqdm.tqdm_notebook(imgPaths[3:6]):
    dist = cv2.imread(imgPath)
    undist = undistorter(imgPath)
    fig, axes = plt.subplots(ncols=2)

    for ax, img in zip(axes, [dist, undist]):
        ax.imshow(img)
        ax.set_xticks([]); ax.set_yticks([])
    fig.suptitle(imgPath)

In [ ]:
i = 3
dist = cv2.imread(imgPaths[i]).mean(-1)
# dist = cv2.GaussianBlur(dist,(3,3),0)
undist = undistorter(imgPaths[i]).mean(-1)
fig, ax = plt.subplots()
ax.imshow(dist - undist);
ax.set_xticks([]); ax.set_yticks([])
ax.set_title('distorted (greyscale) minus undistorted (greyscale)');

# GoPro "WellsFargo"

In [ ]:
ls ~/data2/cameraCalibration/*.MP4

In [ ]:
fpath = '/home/tsbertalan/data2/cameraCalibration/GOPR0019.avi'

In [ ]:
import os, sys
os.listdir(os.path.dirname(fpath))

In [ ]:
from IPython.display import HTML
def showVid(fpath):
    return HTML("""
    <video width=100%% controls autoplay loop>
      <source src="%s" type="video/mp4">
    </video>
    """ % fpath)

In [ ]:
import sys
import skvideo.io
reader = skvideo.io.FFmpegReader(fpath)

In [ ]:
frames = [f for f in tqdm.tqdm_notebook(reader.nextFrame(), total=reader.inputframenum)]

In [ ]:
undistorter = Undistorter()
undistorter.fit(frames[::20])

In [ ]:
dist = frames[400]
udist = undistorter(dist)
fig, axes = plt.subplots(ncols=2)
for ax, frame, title in zip(axes, [dist, udist], ['original', 'undistorted']):
    ax.imshow(frame)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)

In [ ]:
from utils import saveVideo

In [ ]:
saveVideo(frames, 'original.mp4')

In [ ]:
import gc; gc.collect()

In [ ]:
saveVideo(
    (undistorter(frame) for frame in frames),
    'undist.mp4'
)